In [ ]:
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn

In [ ]:
import sys 
print(sys.executable)

In [ ]:
import torch, random, numpy as np, matplotlib.pyplot as plt, os, pickle, pandas as pd
from IPython.display import clear_output

In [ ]:
os.system('nvidia-smi -q -d Memory |grep -A6 GPU|grep Free >tmp')
memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
gpu_number =  int(np.argmax(memory_available))
torch.cuda.set_device(gpu_number)
print(f"Cuda:{gpu_number}")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

Set dataset and parameters

In [ ]:
dataset_type = "DailySports" #RotatedMNIST, FEMNIST, CIFAR10C, DailySports, WISDM

import sys, os
sys.path.insert(1, "./" + dataset_type)
import data, params, models
from utils import write_in_file, plot_avg_acc, plot_worst_acc
from trainingUMA import UMA, adapt_evaluate_dann, adapt_evaluate_mmd, eval_domains
from trainingARM import ARM_CML, train, test

# For reproducibility
torch.random.manual_seed(params.seed)
random.seed(params.seed)
np.random.seed(params.seed)

In [ ]:
for it in range(3):
    clear_output(wait=True)
    PATH = "./" + dataset_type + "/Results" + str(it+1) + "/"
    if not os.path.exists(PATH): os.makedirs(PATH)
    print("DATASET") 
    if dataset_type == "RotatedMNIST":
        domains_train_idxs = random.sample(range(params.nb_domains_tot), params.nb_domains_train)
        domains_test_idxs = list(set(range(params.nb_domains_tot)) - set(domains_train_idxs))
        #domains_train_idxs = pd.read_pickle(PATH+"domains_train")
        #domains_test_idxs = pd.read_pickle(PATH+"domains_test")
        rotatedMnist = data.RotatedMNIST(print_output=False)
        domains_train, domains_test = data.domain_generator(rotatedMnist, domains_train_idxs, domains_test_idxs)
        dataset_train = data.RotatedMNIST(domains_idxs = domains_train_idxs, split='train', print_output=False)
        dataset_test = data.RotatedMNIST(domains_idxs = domains_test_idxs, split='test', print_output=False)    
        write_in_file(domains_train_idxs, PATH+"domains_train")
        write_in_file(domains_test_idxs, PATH+"domains_test")
    elif dataset_type == "FEMNIST":
        dataset_train = data.FEMNISTDataset(split="train")
        domains_train = data.domain_generator(dataset_train, split="train")
        dataset_test = data.FEMNISTDataset(split="test")
        domains_test = data.domain_generator(dataset_test)
    elif dataset_type == "CIFAR10C":
        dataset_train = data.CIFARDataset(split='train')
        domains_train = data.domain_generator(dataset_train)
        dataset_test = data.CIFARDataset(split='test')
        domains_test = data.domain_generator(dataset_test, test='True')
    elif dataset_type == "DailySports":
        root_dir = "DailySports/"
        #domains_train_idxs = random.sample(range(params.nb_domains_tot), params.nb_domains_train)
        #domains_test_idxs = list(set(range(params.nb_domains_tot)) - set(domains_train_idxs))
        domains_train_idxs = pd.read_pickle(PATH+"domains_train")
        domains_test_idxs = pd.read_pickle(PATH+"domains_test")
        HAR_Dataset = data.HARDataset(root_dir, feature_reduction=params.feature_reduction, extract_features=params.extract_features)
        domains_train, domains_test = data.domain_generator(HAR_Dataset, domains_train_idxs, domains_test_idxs)
        dataset_train = data.HARDataset(root_dir, domains_train_idxs, feature_reduction=params.feature_reduction, extract_features=params.extract_features)
        dataset_test = data.HARDataset(root_dir, domains_test_idxs, feature_reduction=params.feature_reduction, extract_features=params.extract_features)
    elif dataset_type == "WISDM":  
        root_dir = "WISDM/"
        domains_total = list(range(params.nb_domains_tot))
        #Remove subjects with corrupted data
        domains_total.remove(14)
        domains_total.remove(18)
        domains_total.remove(42)
        domains_train_idxs = random.sample(domains_total, params.nb_domains_train)
        domains_test_idxs = list(set(domains_total)-set(domains_train_idxs))
        WISDM_Dataset = data.WISDMDataset(root_dir, domains_total, datatype = params.data_type, extract_features=params.extract_features)
        domains_train, domains_test = data.domain_generator(WISDM_Dataset, domains_train_idxs, domains_test_idxs)
        dataset_train = data.WISDMDataset(root_dir, domains_train_idxs, datatype=params.data_type, extract_features=params.extract_features)
        dataset_test = data.WISDMDataset(root_dir, domains_test_idxs, datatype=params.data_type, extract_features=params.extract_features)
        
    train_loader = data.get_loader(dataset_train, sampler_type='group', uniform_over_groups=1,
                              meta_batch_size=params.meta_batch_size,
                              support_size=params.support_size,
                              shuffle=True,
                              pin_memory=True, num_workers=8)
    test_loader = data.get_loader(dataset_test, sampler_type='group', uniform_over_groups=False,
                              meta_batch_size=params.meta_batch_size,
                              support_size=params.support_size,
                              shuffle=False,
                              pin_memory=True, num_workers=8)  

    print("MODELS-UMA")
    #UMA models
    model_dann = models.DANNModel().to(DEVICE)
    uma_dann = UMA(model_dann, params.loss.to(DEVICE), params.lr_sgd_cc, params.lr_sgd_dd, params.lr_adam, adapt_steps=params.ADAPT_STEPS,  model_type = 'dann', adaptive_lr=True, batch_norm=params.batch_norm).fit_dann(domains_train, alpha=params.alpha_, steps=params.training_steps)
    torch.save(uma_dann, PATH+"model_dann.pkl")
#    uma_dann = torch.load(PATH+"model_dann.pkl").to(DEVICE)

    model_mmd = models.MMDModel().to(DEVICE)
    uma_mmd = UMA(model_mmd, params.loss.to(DEVICE), params.lr_sgd_cc, params.lr_sgd_dd, params.lr_adam, adapt_steps=params.ADAPT_STEPS, model_type = 'mmd', adaptive_lr=True, sigma = params.sigma_, batch_norm=params.batch_norm).fit_mmd(domains_train, steps=params.training_steps)
    torch.save(uma_mmd, PATH+"model_mmd.pkl")
#    uma_mmd = torch.load(PATH+"model_mmd.pkl").to(DVEICE)
    
    print("MODELS-SCRATCH")
    #models from scratch
    model_dann_scratch = models.DANNModel().to(DEVICE) 
    model_mmd_scratch = models.MMDModel().to(DEVICE)
    
    print("MODELS-ARM")
    # ARM CML
    context_net = models.ContextNet(in_dim=params.input_dim, out_dim=params.output_dim, hidden_dim=params.hidden_dim_context).to(DEVICE)
    model_arm = models.ARMNet(in_dim=params.input_dim_arm, hidden_dim=params.hidden_dim_net, n_classes=params.num_classes).to(DEVICE)
    algorithm = ARM_CML(model_arm, context_net, params.input_dim, params.loss.to(DEVICE), learning_rate=params.learning_rate, weight_decay=params.weight_decay, optimizer=params.optimizer, momentum=params.momentum, support_size=params.support_size, device=DEVICE, adapt_bn=params.adapt_bn, img_dataset=params.img_dataset)
    history_arm = train(algorithm, train_loader, test_loader, epochs = params.epochsARM)
    torch.save(algorithm, PATH+"model_arm.pkl")
#    algorithm = torch.load(PATH+"model_arm.pkl").to(DEVICE)
    
    print("PERFORMANCE")
    worst_acc_umaDANN, avg_acc_umaDANN, testacc_umaDANN = eval_domains(uma_dann.model, domains_train, domains_test, params.loss.to(DEVICE), uma_dann.lr_sgd_cc, uma_dann.lr_sgd_dd, steps = params.test_steps, alpha = params.alpha_, model_type='dann', batch_norm=params.batch_norm, num_comparison=params.num_comparison, save_output=PATH+"perf_umaDANN")
    worst_acc_scratchDANN, avg_acc_scratchDANN, testacc_scratchDANN = eval_domains(model_dann_scratch, domains_train, domains_test, params.loss.to(DEVICE), uma_dann.lr_sgd_cc, uma_dann.lr_sgd_dd, steps = params.test_steps, alpha = params.alpha_, model_type='dann', batch_norm=params.batch_norm, num_comparison=params.num_comparison, save_output=PATH+"perf_scratchDANN")
    worst_acc_umaMMD, avg_acc_umaMMD, testacc_umaMMD = eval_domains(uma_mmd.model, domains_train, domains_test, params.loss.to(DEVICE), uma_mmd.lr_sgd_cc, uma_mmd.lr_sgd_dd, steps = params.test_steps, sigma=params.sigma_, model_type='mmd', batch_norm=params.batch_norm, num_comparison=params.num_comparison, save_output=PATH+"perf_umaMMD")
    worst_acc_scratchMMD, avg_acc_scratchMMD, testacc_scratchMMD = eval_domains(model_mmd_scratch, domains_train, domains_test, params.loss.to(DEVICE), uma_mmd.lr_sgd_cc, uma_mmd.lr_sgd_dd, steps = params.test_steps, sigma=params.sigma_, model_type='mmd', batch_norm=params.batch_norm, num_comparison=params.num_comparison, save_output=PATH+"perf_scratchMMD")

    plot_avg_acc(params.test_steps, avg_acc_umaDANN, avg_acc_umaMMD, avg_acc_scratchDANN, avg_acc_scratchMMD, save_file=PATH+"average_accuracy.png")
    plot_worst_acc(params.test_steps, worst_acc_umaDANN, worst_acc_umaMMD, worst_acc_scratchDANN, worst_acc_scratchMMD, save_file=PATH+"worst_accuracy.png")
    
    worst_case_acc, average_case_acc, stats = test(algorithm, test_loader, n_samples_per_group=params.n_samples_test, save_output=PATH+"perf_ARM")
 
    print("#SAMPLES COMPARISON")
    n_samples = [50, 100, 145]
    acc_umaDANN_samples, acc_umaMMD_samples, acc_scratchDANN_samples, acc_scratchMMD_samples, acc_ARMCML_samples=[], [], [], [], []
    for s in n_samples:
        print(s)
        seed_ = random.randint(0, 100)
        _, avg_acc_umaDANN_tmp, _ = eval_domains(uma_dann.model, domains_train, domains_test, params.loss.to(DEVICE), uma_dann.lr_sgd_cc, uma_dann.lr_sgd_dd, steps = params.test_steps, alpha = params.alpha_, model_type='dann', batch_norm=params.batch_norm, test_size = s, seed = seed_)
        _, avg_acc_umaMMD_tmp, _ = eval_domains(uma_mmd.model, domains_train, domains_test, params.loss.to(DEVICE), uma_mmd.lr_sgd_cc, uma_mmd.lr_sgd_dd, steps = params.test_steps, sigma=params.sigma_, model_type='mmd', batch_norm=params.batch_norm, test_size = s, seed = seed_)

        _, avg_acc_scratchDANN_tmp, _ = eval_domains(model_dann_scratch, domains_train, domains_test, params.loss.to(DEVICE), uma_dann.lr_sgd_cc, uma_dann.lr_sgd_dd, steps = params.test_steps, alpha = params.alpha_, model_type='dann', batch_norm=params.batch_norm, test_size = s, seed = seed_)
        _, avg_acc_scratchMMD_tmp, _ = eval_domains(model_mmd_scratch, domains_train, domains_test, params.loss.to(DEVICE), uma_mmd.lr_sgd_cc, uma_mmd.lr_sgd_dd, steps = params.test_steps, sigma=params.sigma_, model_type='mmd', batch_norm=params.batch_norm, test_size = s, seed = seed_)
        _, avg_acc_ARMCML_tmp, _ = test(algorithm, test_loader, n_samples_per_group = s)

        acc_umaDANN_samples.append(avg_acc_umaDANN_tmp[-1])
        acc_umaMMD_samples.append(avg_acc_umaMMD_tmp[-1])
        acc_scratchDANN_samples.append(avg_acc_scratchDANN_tmp[-1])
        acc_scratchMMD_samples.append(avg_acc_scratchMMD_tmp[-1])
        acc_ARMCML_samples.append(avg_acc_ARMCML_tmp)
        
    write_in_file(acc_umaDANN_samples, PATH+"acc_umaDANN_samples")
    write_in_file(acc_umaMMD_samples, PATH+"acc_umaMMD_samples")
    write_in_file(acc_scratchDANN_samples, PATH+"acc_scratchDANN_samples")
    write_in_file(acc_scratchMMD_samples, PATH+"acc_scratchMMD_samples")
    write_in_file(acc_ARMCML_samples, PATH+"acc_ARMCML_samples")

In [ ]:
print("AVERAGE ACCURACY")
print(df_average_mean)
#print(df_average_std)

In [ ]:
print("WORST-CASE ACCURACY")
print(df_worse_mean)
#print(df_worse_std)

In [ ]:
#Plot
models_list = ["perf_umaDANN", "perf_umaMMD", "perf_scratchDANN", "perf_scratchMMD"]
average_accuracy = []
std_error = []
for perf in models_list:
    avg_acc = []
    for i in range(3):
        file_name = "./"+dataset_type+"/Results"+str(i+1)+"/"+perf
        avg_acc.append(list(pd.read_pickle(file_name)['average accuracy']))
    average_accuracy.append(np.mean(np.array(avg_acc),0))
    std_error.append(np.std(np.array(avg_acc),0))

In [ ]:
plt.plot(range(params.test_steps), average_accuracy[0], c='tab:blue', label="UMA-DANN")
plt.fill_between(range(params.test_steps), average_accuracy[0]-std_error[0], average_accuracy[0]+std_error[0], color='tab:blue', alpha=0.2)
plt.plot(range(params.test_steps), average_accuracy[1], c='tab:green', label="UMA-MMD")
plt.fill_between(range(params.test_steps), average_accuracy[1]-std_error[1], average_accuracy[1]+std_error[1], color='tab:green', alpha=0.2)
plt.plot(range(params.test_steps), average_accuracy[2], c='tab:orange', label="Scratch-DANN")
plt.fill_between(range(params.test_steps), average_accuracy[2]-std_error[2], average_accuracy[2]+std_error[2], color='tab:orange', alpha=0.2)
plt.plot(range(params.test_steps), average_accuracy[3], c='tab:purple', label="Scratch-MMD")
plt.fill_between(range(params.test_steps), average_accuracy[3]-std_error[3], average_accuracy[3]+std_error[3], color='tab:purple', alpha=0.2)
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.legend()
plt.legend(loc=4)
plt.title("Average accuracy", fontsize=12)
plt.savefig("./" + dataset_type + "/average_accuracy_std.png")

In [ ]:
#Comparison with ARM-CML
models_list = ["perf_ARM", "perf_umaDANN", "perf_umaMMD"]

df_mean = pd.DataFrame(columns=["Avg 200 steps", "Worse 200 steps"], index=models_list)
df_std = pd.DataFrame(columns=["Avg 200 steps", "Worse 200 steps"], index=models_list)

for perf in models_list:
    df = pd.DataFrame(columns=["Avg 200 steps", "Worse 200 steps"])
    for i in range(3):
        file_name = "./"+dataset_type+"/Results"+str(i+1)+"/"+perf
        objects = pd.read_pickle(file_name)
        if perf == "perf_ARM":
            df.loc[len(df)] = [objects["average_acc"], objects["worst_case_acc"]]
        else:
            df.loc[len(df)] = [objects["average accuracy"][-1], objects["worst_case_accuracy"][-1]]
    df_mean.loc[perf]=[df["Avg 200 steps"].mean(), df["Worse 200 steps"].mean()]
    df_std.loc[perf]=[df["Avg 200 steps"].std(), df["Worse 200 steps"].std()]

In [ ]:
print("AVERAGE ACCURACY")
print(df_mean)
#print(df_std)

In [ ]:
#Vary number of unlabeled samples at test time
models_list = ["acc_scratchDANN_samples", "acc_scratchMMD_samples", "acc_ARMCML_samples", "acc_umaDANN_samples", "acc_umaMMD_samples"]

df_mean = pd.DataFrame(columns=["50 examples", "100 examples", "150 examples"], index=models_list)
df_std = pd.DataFrame(columns=["50 examples", "100 examples", "150 examples"], index=models_list)

for perf in models_list:
    df = pd.DataFrame(columns=["50 examples", "100 examples", "150 examples"])
    for i in range(3):
        file_name = "./"+dataset_type+"/Results"+str(i+1)+"/"+perf
        objects = pd.read_pickle(file_name)
        df.loc[len(df)] = [objects[0], objects[1], objects[2]]
    df_mean.loc[perf]=[df["50 examples"].mean(), df["100 examples"].mean(), df["150 examples"].mean()]
    df_std.loc[perf]=[df["50 examples"].std(), df["100 examples"].std(), df["150 examples"].std()]

In [ ]:
print("AVERAGE ACCURACY")
print(df_mean)
#print(df_std)